In [2]:
import ollama
import json

In [6]:
def generate(prompt: str):
    with open("prompt/system_prompt.txt", 'r') as f:
        system_prompt = f.read()

    response = ollama.chat(
        model='deepseek-r1:8b',
        messages=[
            {"role": "user", "content": prompt},
            {"role": "system", "content": system_prompt}
        ]
    )

    content = response['message']['content']
    # content = content[::-1]
    # start = content.find(''.join(reversed('</answer>'))) + len(''.join(reversed('?</answer>')))
    # end = start + content[start:].find(''.join(reversed('<answer>')))
    # content = content[start:end]
    # content = content[::-1]

    return content

In [10]:
question1 = generate("""
                     A
                     """)

In [11]:
print(question1)

<think>
First, the system prompt says I'm the INTERVIEWER, and I must follow a sequence strictly. The type of interview isn't specified here; it's just given as an example that there are two types: generic (HR) or technical.

In this user message, it seems like they're providing instructions for how to format my responses, but in the context, it might be part of the conversation. Looking back:

User said: "A" – which is probably a response from a candidate or something similar. But according to the rules, I need to start by greeting if it's a generic interview.

The system prompt says: "MUST -> The type of interview will be mentioned in starting and there are two type of interviews first is generic (HR) and second is technical."

But in this interaction, no type is specified. Perhaps I need to assume the type or ask for clarification? No, let's read carefully.

The user message is: "A" – that might be incomplete or a placeholder. Looking at the conversation history:

There was a previo

In [ ]:
name = input("Name")
role = input("Role")
experience  = input("Years of Exp")
prompt = str({'Name': {name}, 'role': {role}, 'years of experience': {experience}})

In [10]:
print(prompt)

{'Name': {'Mohit'}, 'role': {'DS'}, 'years of experience': {'0-1'}}


In [1]:
def generatetechnical(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    try:
        response = ollama.chat(
            model='phi3:3.8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )
        
        content = response['message']['content']

        tags = ("question", "A", "B", "C", "D")
        qna = {}
        for tag in tags:
            start_tag = "<" + tag + ">"
            end_tag = "</" + tag + ">"
            material = content[::-1]
            
            start_idx = material.find(''.join(reversed(end_tag))) + len(''.join(reversed(end_tag)))
            end_idx = start_idx + material[start_idx:].find(''.join(reversed(start_tag)))
            item = material[start_idx:end_idx]
            item = material[::-1]

            qna[tag] = item

        return qna

    except Exception as e:
        return f"Error: Failed to generate question: {str(e)}"

In [19]:
def generatetechnical(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    try:
        response = ollama.chat(
            model='phi3:3.8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )
        
        content = response['message']['content']['question']

        tags = ("question", "A", "B", "C", "D")
        qna = {}
        material = content[::-1]        
        for tag in tags:
            if tag != "question":
                start_tag = "<option>" + tag + ")"
                end_tag = "</option>"
                material = content[::-1]
            else:    
                start_tag = "<question>"
                end_tag = "</question>"
                material = content[::-1]

            start_idx = material.find(''.join(reversed(end_tag))) + len(''.join(reversed(end_tag)))
            end_idx = start_idx + material[start_idx:].find(''.join(reversed(start_tag)))
            item = material[start_idx:end_idx]
            item = material[::-1]

            qna[tag] = item

        return qna
    except Exception as e:
        return f"Error: Failed to generate question: {str(e)}"

In [21]:
response = generatetechnical(prompt)

In [27]:
with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
    system_prompt = f.read()
    
response = ollama.chat(
            model='deepseek-r1:8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )
        
content = response['message']['content']

In [35]:
qna = {}
tags = ("question", "options")
material = content[::-1]     
for tag in tags:
    start_tag = "<" + tag + ">"
    end_tag = "</" + tag + ">"
    start_idx = material.find(''.join(reversed(end_tag))) + len(''.join(reversed(end_tag)))
    end_idx = start_idx + material[start_idx:].find(''.join(reversed(start_tag)))
    item = material[start_idx:end_idx]
    item = item[::-1]
    qna[tag] = item

options = ("A:", "B:", "C:", "D:")
indices = []
for option in options:
    indices.append(qna["options"].find(option))
indices.append(len(qna["options"]) - 1)
answers = []
for i in range(4):
    answers.append(qna["options"][indices[i]: indices[i+1]])

In [39]:
answers[0]

'A: Regression predicts continuous values; classification predicts discrete categories.  \n'

In [33]:
content

"<think>\nFirst, I am considering that I'm the interviewer. The user has provided their name (Mohit), role (Data Science Intern), and years of experience (0-1). That's clear enough.\n\nSince this is an entry-level position with 0-1 year experience, my questions should focus on fundamentals in data science. Key areas include basic concepts like machine learning algorithms, statistics, Python libraries for data manipulation (e.g., Pandas, NumPy), and perhaps some SQL basics since it's common in data roles.\n\nNow, I need to ask one multiple-choice question at a time. Each question must be concise: 60-100 characters excluding tags. Options should be exactly four, with only one correct answer marked internally.\n\nI'll start by asking a fundamental question related to their role. For example, something about regression vs. classification in machine learning, which is core for data science interns.\n\nLet me think of the first question:\n\nQuestion: What is the main difference between regre

In [ ]:
content

In [11]:
response["question"]

'As an interviewer for Mohit, who is currently working as a Data Science Intern with no more than one year of experience:\n\n<question>Which library in Python would you use primarily for data manipulation and analysis?</question>\n<options><option>A) Matplotlib - Plotting graphs.</option>\n  <option>B) NumPy - Array operations.</option>\n  <option>C) Pandas - Data structures and data analysis tools (Correct Answer).</option>\n  <option>D) Scikit-learn - Machine learning models.</option></options>\n\n(Awaiting candidate\'s response...)\n\nIf the correct answer is provided: "3/20" or similar. Evaluate his strength in data manipulation and analysis, suggesting further discussion on Python for Data Science to assess deeper understanding of tools like Pandas within a technical interview setting based on Mohit’s performance throughout all 20 questions covering coding concepts, tool usage, problem-solving abilities specific to the role.'

In [72]:
def generatetechnical0(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    try:
        response = ollama.chat(
            model='phi3:3.8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )
        
        content = response['message']['content']

        qna = {}
        tags = ("question", "options")
        material = content[::-1]     
        for tag in tags:
            start_tag = "<" + tag + ">"
            end_tag = "</" + tag + ">"
            start_idx = material.find(''.join(reversed(end_tag))) + len(''.join(reversed(end_tag)))
            end_idx = start_idx + material[start_idx:].find(''.join(reversed(start_tag)))
            item = material[start_idx:end_idx]
            item = item[::-1]
            qna[tag] = item

        options = ("A.", "B.", "C.", "D.")
        indices = []
        for option in options:
            indices.append(qna["options"].find(option))
        indices.append(len(qna["options"]) - 1)
        answers = [qna["question"]]
        for i in range(4):
            answers.append(qna["options"][indices[i]: indices[i+1]])


        return content, answers

    except Exception as e:
        return f"Error: Failed to generate question: {str(e)}"

In [108]:
response0 = generatetechnical0(prompt)

In [110]:
type(response0)

tuple

In [113]:
response0[0]

'```json\n{\n    "question": "<response>What is Pandas used for in Python?",\n    "options": {\n        "A": "Data manipulation and analysis.",\n        "B": "Web page design.",\n        "C": "Mobile app development.",\n        "D": "Network security protocols."\n    },\n    "answer": "A"\n}\n```'

In [61]:
def generatetechnical(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    try:
        response = ollama.chat(
            model='deepseek-r1:8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )
        
        content = response['message']['content']

        qna = {}
        tags = ("question", "options")
        material = content[::-1]     
        for tag in tags:
            start_tag = "<" + tag + ">"
            end_tag = "</" + tag + ">"
            start_idx = material.find(''.join(reversed(end_tag))) + len(''.join(reversed(end_tag)))
            end_idx = start_idx + material[start_idx:].find(''.join(reversed(start_tag)))
            item = material[start_idx:end_idx]
            item = item[::-1]
            qna[tag] = item

        options = ("A.", "B.", "C.", "D.")
        indices = []
        for option in options:
            indices.append(qna["options"].find(option))
        indices.append(len(qna["options"]) - 1)
        answers = [qna["question"]]
        for i in range(4):
            answers.append(qna["options"][indices[i]: indices[i+1]])


        return content, answers

    except Exception as e:
        return f"Error: Failed to generate question: {str(e)}"

In [ ]:
response = generatetechnical(prompt)
start_idx = response[0].find("</think>") + len("</think>")
a = ''.join(response[0][start_idx:].split('\n'))
final = json.loads(a)

In [127]:
final

{'question': 'To load data from a CSV file into a Pandas DataFrame, which method do you use?',
 'options': {'A': 'pd.read_csv()',
  'B': 'numpy.loadtxt()',
  'C': 'pandas.import_csv()',
  'D': "dataframe.load('csv')"},
 'answer': 'A'}

In [4]:
import ollama
import json


def generatetechnical(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    response = ollama.chat(
        model='deepseek-r1:8b',
        messages=[
            {"role": "user", "content": prompt},
            {"role": "system", "content": system_prompt}
        ]
    )
    
    content = response['message']['content']
    start_idx = content.find("</think>") + len("</think>")
    a = ''.join(content[start_idx:].split('\n'))
    final = json.loads(a)
    return content, final

In [11]:
type(response[0])

str

In [12]:
def generatetechnical(prompt: str):
    with open("prompt/system_prompt_mcq.txt", mode='r', encoding="utf-8", errors="ignore") as f:
        system_prompt = f.read()

    try:
        response = ollama.chat(
            model='deepseek-r1:8b',
            messages=[
                {"role": "user", "content": prompt},
                {"role": "system", "content": system_prompt}
            ]
        )    
        content = response['message']['content']
        start_idx = content.find("</think>") + len("</think>")
        a = ''.join(content[start_idx:].split('\n'))
        final = json.loads(a)
        return final

    except Exception as e:
        return f"Error: Failed to generate mcq (error while generating): {str(e)}"

In [13]:
name = input("Name")
role = input("Role")
experience  = input("Years of Exp")
prompt = str({'Name': {name}, 'role': {role}, 'years of experience': {experience}})

In [14]:
response = generatetechnical(prompt)

In [17]:
response

{'question': 'In machine learning, what is the primary goal of supervised learning?',
 'options': {'A': 'To predict unknown future events based on learned patterns from labeled data',
  'B': 'To cluster unlabeled data into groups without predefined categories',
  'C': 'To find relationships between variables using statistical methods',
  'D': 'To train a model to generate outputs for inputs it has never seen, but with no guidance'},
 'answer': 'A'}